# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770762260936                   -0.53    8.0         
  2   -2.772140288493       -2.86       -1.31    1.0    175ms
  3   -2.772169997173       -4.53       -2.58    1.0    153ms
  4   -2.772170711632       -6.15       -3.62    2.0    217ms
  5   -2.772170722632       -7.96       -4.18    2.0    175ms
  6   -2.772170722994       -9.44       -5.16    1.0    185ms
  7   -2.772170723015      -10.67       -6.27    2.0    177ms
  8   -2.772170723015      -13.74       -6.67    1.0    173ms
  9   -2.772170723015      -13.95       -7.88    2.0    191ms
 10   -2.772170723015      -14.15       -8.68    2.0    200ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770852912772                   -0.52    9.0         
  2   -2.772063728262       -2.92       -1.32    1.0    169ms
  3   -2.7

1.7735581061500645

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770806375533                   -0.53    9.0         
  2   -2.772049297135       -2.91       -1.30    1.0    147ms
  3   -2.772082359196       -4.48       -2.65    1.0    165ms
  4   -2.772083412908       -5.98       -3.77    2.0    201ms
  5   -2.772083417367       -8.35       -4.23    2.0    171ms
  6   -2.772083417803       -9.36       -5.11    1.0    175ms
  7   -2.772083417810      -11.13       -6.48    1.0    161ms
  8   -2.772083417811      -12.60       -6.23    2.0    193ms
  9   -2.772083417811      -13.57       -7.29    1.0    183ms
 10   -2.772083417811   +  -14.12       -7.40    1.0    172ms
 11   -2.772083417811      -13.83       -7.83    1.0    175ms
 12   -2.772083417811   +  -14.05       -8.28    2.0    193ms

Polarizability via ForwardDiff:       1.7725349819449114
Polarizability via finite difference: 1.7735581061500645
